In [19]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 11
### Team Member Names: Akram, Annie, Jester
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [20]:
# Important Constants: 
amount = 1_000_000 # Initial investment amount of $1,000,000
group = 11
start_date, end_date = '2022-09-30', '2024-09-30'
min_stocks, max_stocks = 12, 24
# Reading in CSV file: 
tickers = pd.read_csv('Tickers.csv')
ticker_lst = list(tickers['Tickers'])

# Initializing variable to store the tickers we will use in our portfolio
columns = ['Ticker', 'Price', 'Currency', 'Shares', 'Value', 'Weight']
Portfolio_Final = pd.DataFrame(columns=columns)

In [21]:
# Filtering valid stocks by inputting a list of strings for each ticker. 
def filter_stocks(ticker_lst):
    valid_tickers, invalid_tickers = {}, []
    # Loop through all tickers to check if they are valid
    for ticker in ticker_lst:
        try:
            stock = yf.Ticker(ticker)
            info = stock.fast_info # Get basic stock info
            hist = stock.history(start=start_date, end=end_date) # Get stock history
            hist.index = hist.index.strftime('%Y-%m-%d')
            avg_volume = hist.loc[((hist.index >= '2023-09-30') & (hist.index <= '2024-09-30'))]['Volume'].mean() # Calculate average volume in specified date range.
            if ((hist.empty is not None) and # filter for stocks delisted on yfinance
                (info.get("currency") == "USD" or info.get("currency") == "CAD") and # filter for stocks that are not USD
                (avg_volume >= 100_000)): # Filter by volume greater than 100,000
                valid_tickers[ticker] = hist['Close'] # Store the close prices of the stock as a Series
            else:
                invalid_tickers.append(ticker)
        except:
            invalid_tickers.append(ticker)
    return [valid_tickers, invalid_tickers]

In [26]:
ticker_lst = filter_stocks(ticker_lst)[0]

data = pd.DataFrame()
for ticker in ticker_lst:
    data[ticker] = ticker_lst[ticker]
# data = yf.download(ticker_lst, period="2y", interval="1d")['Close']
# data.index = data.index.strftime('%Y-%m-%d')
returns = data.pct_change()
returns.drop(index=returns.index[0], inplace = True)

data

/var/folders/9g/h_n63rvn21s1k0y3xvf0n28h0000gn/T/ipykernel_22039/1737150026.py:8: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change()


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-30,136.535339,123.195908,92.569466,248.165283,45.381317,113.000000,130.871185,121.080002,28.553211,6.51,...,107.784981,113.588989,37.189999,24.323668,76.140366,144.759399,490.247040,185.840042,148.646622,36.471252
2022-10-03,140.734161,126.968582,95.209930,255.485870,47.006172,115.879997,135.828186,126.050003,29.394680,6.53,...,111.830017,114.575356,37.799999,25.033070,77.623253,149.491776,500.410339,191.029297,150.072906,37.601929
2022-10-04,144.340179,130.337402,98.615753,264.571411,49.424355,121.089996,141.105392,133.509995,30.614340,6.74,...,116.743210,117.543640,42.610001,25.316835,79.079201,154.458008,507.845917,195.913315,155.409973,38.560749
2022-10-05,144.636536,131.567444,98.902748,264.600372,49.032467,120.949997,140.047958,132.110001,30.179422,6.67,...,119.147339,117.041313,41.959999,25.104010,78.620850,156.936478,511.631805,191.372696,154.526611,38.171799
2022-10-06,143.678223,128.776917,98.013031,259.903259,48.516335,120.300003,138.117554,132.199997,29.744503,6.64,...,119.519409,113.762520,41.320000,24.421211,75.751678,155.758026,503.924347,189.073776,152.382599,37.158707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-23,226.221115,192.294418,114.271446,337.834869,74.300003,193.880005,266.603333,156.300003,39.869999,3.31,...,165.960007,164.996796,108.230003,22.520000,85.299362,202.516693,573.409973,244.000000,128.740005,45.355824
2024-09-24,227.120117,191.976959,112.808334,338.243225,73.419998,193.960007,265.516205,155.809998,39.450001,3.29,...,166.949997,164.828186,107.959999,22.410000,84.736038,203.599564,575.190002,248.960007,129.520004,45.138241
2024-09-25,226.121216,189.744781,112.081764,335.683624,73.019997,192.529999,265.476318,152.220001,39.250000,3.23,...,168.270004,165.334000,106.889999,22.250000,83.826813,203.718765,576.309998,245.500000,131.509995,44.258026


In [ ]:
# Code to output final dataframe to a CSV file called Stocks_Group_XX.csv
Stocks_Final = Portfolio_Final[['Ticker', 'Shares']]
Stocks_Final.to_csv(f'Stocks_Group_{group}.csv', index=False)

In [ ]:
# def options_chain(symbol):

#     tk = yf.Ticker(symbol)
#     # Expiration dates
#     exps = tk.options

#     # Get options for each expiration
#     options = pd.DataFrame()
#     for e in exps:
#         opt = tk.option_chain(e)
#         opt = pd.DataFrame().append(opt.calls).append(opt.puts)
#         opt['expirationDate'] = e
#         options = options.append(opt, ignore_index=True)

#     # Bizarre error in yfinance that gives the wrong expiration date
#     # Add 1 day to get the correct expiration date
#     options['expirationDate'] = pd.to_datetime(options['expirationDate']) + datetime.timedelta(days = 1)
#     options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
#     # Boolean column if the option is a CALL
#     options['CALL'] = options['contractSymbol'].str[4:].apply(
#         lambda x: "C" in x)
    
#     options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
#     options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
#     # Drop unnecessary and meaningless columns
#     options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

#     return options

In [ ]:
aapl = yf.Ticker('XIU')
# for exp in aapl.options:
#     aapl.option_chain(exp)

# aapl.option_chain(aapl.options[0])
len(aapl.options)

0

In [ ]:
Portfolio_Final

,Ticker,Price,Currency,Shares,Value,Weight


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

---
<p style="color: #004dd3">
Akram Jamil
</p>

<p style="color: #2C8CA9">
Jester Yang
</p>

<p style="color: #3cc19d;">
Annie Wong
</p>

---